In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import warnings
import os
import datetime

# Suppress warnings
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported successfully!")


2025-04-07 20:59:34.224799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-07 20:59:34.245678: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-07 20:59:34.245728: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 20:59:34.259355: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-07 20:59:35.001525: W tensorflow/compiler/tf

Libraries imported successfully!


In [2]:
def create_fuzzy_system():
    """
    Create and return a fuzzy logic control system for load forecasting.
    """
    try:
        # Define input and output variables
        time_of_day = ctrl.Antecedent(np.arange(0, 24, 0.1), 'time_of_day')
        previous_load = ctrl.Antecedent(np.arange(0, 20000, 100), 'previous_load')
        forecasted_load = ctrl.Consequent(np.arange(0, 20000, 100), 'forecasted_load')

        # Define fuzzy sets for time of day
        time_of_day['midnight'] = fuzz.trimf(time_of_day.universe, [0, 3, 6])
        time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [5, 8, 11])
        time_of_day['noon'] = fuzz.trimf(time_of_day.universe, [10, 12, 14])
        time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [13, 15, 17])
        time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [16, 19, 22])
        time_of_day['night'] = fuzz.trimf(time_of_day.universe, [21, 23, 24])

        # Define fuzzy sets for previous load
        previous_load['very_low'] = fuzz.trimf(previous_load.universe, [0, 2000, 4000])
        previous_load['low'] = fuzz.trimf(previous_load.universe, [3000, 5000, 7000])
        previous_load['average'] = fuzz.trimf(previous_load.universe, [6000, 8000, 10000])
        previous_load['high'] = fuzz.trimf(previous_load.universe, [9000, 12000, 15000])
        previous_load['very_high'] = fuzz.trimf(previous_load.universe, [14000, 17000, 20000])

        # Define fuzzy sets for forecasted load
        forecasted_load['very_low'] = fuzz.trimf(forecasted_load.universe, [0, 2000, 4000])
        forecasted_load['low'] = fuzz.trimf(forecasted_load.universe, [3000, 5000, 7000])
        forecasted_load['average'] = fuzz.trimf(forecasted_load.universe, [6000, 8000, 10000])
        forecasted_load['high'] = fuzz.trimf(forecasted_load.universe, [9000, 12000, 15000])
        forecasted_load['very_high'] = fuzz.trimf(forecasted_load.universe, [14000, 17000, 20000])

        # Define fuzzy rules
        rule1 = ctrl.Rule(time_of_day['midnight'] & previous_load['very_low'], forecasted_load['very_low'])
        rule2 = ctrl.Rule(time_of_day['midnight'] & previous_load['low'], forecasted_load['low'])
        rule3 = ctrl.Rule(time_of_day['morning'] & previous_load['average'], forecasted_load['high'])
        rule4 = ctrl.Rule(time_of_day['noon'] & previous_load['high'], forecasted_load['very_high'])
        rule5 = ctrl.Rule(time_of_day['afternoon'] & previous_load['high'], forecasted_load['high'])
        rule6 = ctrl.Rule(time_of_day['evening'] & previous_load['average'], forecasted_load['high'])
        rule7 = ctrl.Rule(time_of_day['night'] & previous_load['low'], forecasted_load['average'])
        rule8 = ctrl.Rule(time_of_day['night'] & previous_load['very_low'], forecasted_load['low'])
        rule9 = ctrl.Rule(time_of_day['morning'] & previous_load['low'], forecasted_load['average'])
        rule10 = ctrl.Rule(time_of_day['noon'] & previous_load['average'], forecasted_load['high'])

        # Create control system
        load_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10])
        load_forecasting = ctrl.ControlSystemSimulation(load_ctrl)
        
        print("Fuzzy logic system created successfully!")
        return load_forecasting
    except Exception as e:
        print(f"Error creating fuzzy system: {e}")
        return None

def predict_load_fuzzy(fuzzy_system, hour, prev_load):
    """
    Predict load using fuzzy logic.
    
    Args:
        fuzzy_system: The fuzzy control system
        hour: Hour of the day (0-23)
        prev_load: Previous load value
        
    Returns:
        Predicted load value
    """
    try:
        # Ensure inputs are within range
        hour = max(0, min(23.9, hour))
        prev_load = max(0, min(19900, prev_load))
        
        # Set inputs
        fuzzy_system.input['time_of_day'] = hour
        fuzzy_system.input['previous_load'] = prev_load
        
        # Compute result
        fuzzy_system.compute()
        
        # Check if forecasted_load is in output
        if 'forecasted_load' in fuzzy_system.output:
            return fuzzy_system.output['forecasted_load']
        else:
            print(f"Warning: forecasted_load not in fuzzy system output. Available keys: {fuzzy_system.output.keys()}")
            # Return a default prediction based on previous load
            return prev_load
    except Exception as e:
        print(f"Error in fuzzy prediction: {e}")
        # Return previous load as fallback
        return prev_load


In [3]:
def create_dataset(dataset, time_step=1):
    """
    Convert an array of values into a dataset matrix for LSTM.
    
    Args:
        dataset: Array of values to be converted
        time_step: Number of time steps to look back
        
    Returns:
        Tuple of X (input) and Y (output) datasets
    """
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

def build_lstm_model(data, state_name, time_step=24):
    """
    Build and train an LSTM model for load forecasting.
    
    Args:
        data: DataFrame containing load data
        state_name: Name of the state to forecast for
        time_step: Number of time steps to look back
        
    Returns:
        Tuple of (trained model, scaler)
    """
    try:
        # Filter data for the specific state
        state_data = data[data['State'] == state_name]['Value'].values
        if len(state_data) == 0:
            print(f"No data found for state: {state_name}")
            print(f"Available states: {data['State'].unique()}")
            return None, None
            
        state_data = state_data.reshape(-1, 1)
        
        # Normalize the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        state_data = scaler.fit_transform(state_data)
        
        # Split into train and test sets
        train_size = int(len(state_data) * 0.8)
        train_data = state_data[0:train_size, :]
        test_data = state_data[train_size:len(state_data), :]
        
        # Reshape into X=t and Y=t+1
        X_train, y_train = create_dataset(train_data, time_step)
        X_test, y_test = create_dataset(test_data, time_step)
        
        # Reshape input to be [samples, time steps, features]
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
        
        # Create the LSTM model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))
        
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Train the model with early stopping
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
        model.fit(
            X_train, y_train,
            batch_size=32,
            epochs=100,
            validation_data=(X_test, y_test),
            verbose=0,
            callbacks=[callback]
        )
        
        # Make predictions
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)
        
        # Invert predictions to original scale
        train_predict = scaler.inverse_transform(train_predict)
        test_predict = scaler.inverse_transform(test_predict)
        y_train_inv = scaler.inverse_transform([y_train])
        y_test_inv = scaler.inverse_transform([y_test])
        
        # Calculate RMSE
        train_rmse = np.sqrt(mean_squared_error(y_train_inv[0], train_predict[:,0]))
        test_rmse = np.sqrt(mean_squared_error(y_test_inv[0], test_predict[:,0]))
        
        # Calculate R² score
        train_r2 = r2_score(y_train_inv[0], train_predict[:,0])
        test_r2 = r2_score(y_test_inv[0], test_predict[:,0])
        
        print(f'State: {state_name}')
        print(f'Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}')
        print(f'Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')
        
        return model, scaler
    
    except Exception as e:
        print(f"Error building LSTM model: {e}")
        return None, None


In [4]:
def hybrid_load_forecast(data, state_name, hour, prev_load, fuzzy_system, time_step=24):
    """
    Forecast load using a hybrid of fuzzy logic and LSTM.
    
    Args:
        data: DataFrame containing load data
        state_name: Name of the state to forecast for
        hour: Hour of the day
        prev_load: Previous load value
        fuzzy_system: Fuzzy logic system
        time_step: Number of time steps for LSTM
        
    Returns:
        Tuple of (hybrid prediction, nn prediction, fuzzy prediction)
    """
    try:
        # Get neural network prediction
        model, scaler = build_lstm_model(data, state_name, time_step)
        
        if model is None or scaler is None:
            print("Neural network model could not be built. Using only fuzzy logic.")
            fuzzy_prediction = predict_load_fuzzy(fuzzy_system, hour, prev_load)
            return fuzzy_prediction, 0, fuzzy_prediction
        
        # Prepare the input sequence for prediction
        recent_data = data[data['State'] == state_name]['Value'].values[-time_step:]
        recent_data = recent_data.reshape(-1, 1)
        recent_data = scaler.transform(recent_data)
        
        # Reshape for LSTM input
        X_input = recent_data.reshape(1, time_step, 1)
        
        # Get LSTM prediction
        nn_prediction = model.predict(X_input)
        nn_prediction = scaler.inverse_transform(nn_prediction)[0, 0]
        
        # Get fuzzy logic prediction
        fuzzy_prediction = predict_load_fuzzy(fuzzy_system, hour, prev_load)
        
        # Combine predictions (weighted average)
        # Give more weight to NN for long-term patterns and fuzzy for interpretability
        hybrid_prediction = (0.7 * nn_prediction + 0.3 * fuzzy_prediction)
        
        return hybrid_prediction, nn_prediction, fuzzy_prediction
    
    except Exception as e:
        print(f"Error in hybrid forecast: {e}")
        # Fallback to fuzzy prediction
        fuzzy_prediction = predict_load_fuzzy(fuzzy_system, hour, prev_load)
        return fuzzy_prediction, 0, fuzzy_prediction


In [5]:
def evaluate_model(data, state_name, fuzzy_system):
    """
    Evaluate the hybrid model for a specific state.
    
    Args:
        data: DataFrame containing load data
        state_name: Name of the state to evaluate
        fuzzy_system: Fuzzy logic system
        
    Returns:
        Tuple of (hybrid predictions, nn predictions, fuzzy predictions)
    """
    try:
        # Get the actual values for comparison
        state_data = data[data['State'] == state_name]
        if len(state_data) == 0:
            print(f"No data found for state: {state_name}")
            return [], [], []
            
        actual_values = state_data['Value'].values[-24:]
        
        # Predictions using hybrid model
        hybrid_predictions = []
        nn_predictions = []
        fuzzy_predictions = []
        
        for i in range(24):
            hour = i
            prev_load = actual_values[i-1] if i > 0 else actual_values[-1]
            
            # Get predictions
            hybrid_pred, nn_pred, fuzzy_pred = hybrid_load_forecast(
                data, state_name, hour, prev_load, fuzzy_system
            )
            
            hybrid_predictions.append(hybrid_pred)
            nn_predictions.append(nn_pred)
            fuzzy_predictions.append(fuzzy_pred)
        
        # Calculate errors
        hybrid_mse = mean_squared_error(actual_values, hybrid_predictions)
        hybrid_rmse = np.sqrt(hybrid_mse)
        hybrid_r2 = r2_score(actual_values, hybrid_predictions)
        
        nn_mse = mean_squared_error(actual_values, nn_predictions)
        nn_rmse = np.sqrt(nn_mse)
        nn_r2 = r2_score(actual_values, nn_predictions)
        
        fuzzy_mse = mean_squared_error(actual_values, fuzzy_predictions)
        fuzzy_rmse = np.sqrt(fuzzy_mse)
        fuzzy_r2 = r2_score(actual_values, fuzzy_predictions)
        
        # Calculate MAPE
        hybrid_mape = np.mean(np.abs((actual_values - hybrid_predictions) / actual_values)) * 100
        nn_mape = np.mean(np.abs((actual_values - nn_predictions) / actual_values)) * 100
        fuzzy_mape = np.mean(np.abs((actual_values - fuzzy_predictions) / actual_values)) * 100
        
        print(f"\nEvaluation Results for {state_name}:")
        print(f"Hybrid Model - RMSE: {hybrid_rmse:.2f}, R²: {hybrid_r2:.4f}, MAPE: {hybrid_mape:.2f}%")
        print(f"Neural Network - RMSE: {nn_rmse:.2f}, R²: {nn_r2:.4f}, MAPE: {nn_mape:.2f}%")
        print(f"Fuzzy Logic - RMSE: {fuzzy_rmse:.2f}, R²: {fuzzy_r2:.4f}, MAPE: {fuzzy_mape:.2f}%")
        
        # Plot results
        plt.figure(figsize=(12, 6))
        plt.plot(actual_values, 'k-', label='Actual Load')
        plt.plot(hybrid_predictions, 'r-', label='Hybrid Prediction')
        plt.plot(nn_predictions, 'g--', label='Neural Network Prediction')
        plt.plot(fuzzy_predictions, 'b--', label='Fuzzy Logic Prediction')
        plt.legend()
        plt.title(f'Load Forecasting for {state_name}')
        plt.xlabel('Hour')
        plt.ylabel('Load (MW)')
        plt.grid(True)
        
        # Save the plot
        output_dir = 'results'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        plt.savefig(f"{output_dir}/{state_name}_forecast.png")
        plt.close()
        
        return hybrid_predictions, nn_predictions, fuzzy_predictions
    
    except Exception as e:
        print(f"Error in model evaluation: {e}")
        return [], [], []


In [6]:
def load_data(file_path):
    """
    Load and preprocess the data.
    
    Args:
        file_path: Path to the data file
        
    Returns:
        Preprocessed DataFrame
    """
    try:
        # Try to load the data
        if file_path.endswith('.csv'):
            data = pd.read_csv(file_path)
        elif file_path.endswith('.txt'):
            data = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated
        else:
            # Try different delimiters
            data = pd.read_csv(file_path, sep=None, engine='python')
        
        print(f"Data loaded successfully with {data.shape[0]} rows and {data.shape[1]} columns.")
        
        # Check if expected columns exist
        if 'Datetime' not in data.columns and 'Date' in data.columns:
            data['Datetime'] = data['Date']
        
        # Convert timestamp to datetime if it exists
        if 'Datetime' in data.columns:
            data['Datetime'] = pd.to_datetime(data['Datetime'])
            data['Hour'] = data['Datetime'].dt.hour
                    # If 'State' column doesn't exist but we have state data in another column
        if 'State' not in data.columns and 'Region' in data.columns:
            data['State'] = data['Region']
        
        # If 'Value' column doesn't exist but we have load data in another column
        if 'Value' not in data.columns:
            # Try to find a column that might contain load values
            numeric_cols = data.select_dtypes(include=['number']).columns
            if len(numeric_cols) > 0:
                # Use the first numeric column that's not Hour
                for col in numeric_cols:
                    if col != 'Hour':
                        print(f"Using column '{col}' as load values.")
                        data['Value'] = data[col]
                        break
        
        # Check if we have the minimum required columns
        required_cols = ['State', 'Value']
        missing_cols = [col for col in required_cols if col not in data.columns]
        
        if missing_cols:
            print(f"Warning: Missing required columns: {missing_cols}")
            # Create dummy columns if needed
            if 'State' not in data.columns:
                print("Creating dummy 'State' column with value 'UNKNOWN'")
                data['State'] = 'UNKNOWN'
            if 'Value' not in data.columns:
                print("Error: No numeric column found for load values.")
                return None
        
        # Add hour column if it doesn't exist
        if 'Hour' not in data.columns:
            print("Warning: No 'Hour' column. Creating dummy hour values.")
            data['Hour'] = np.arange(len(data)) % 24
        
        return data
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None



In [7]:
def forecast_all_states(data, fuzzy_system, hour_to_forecast=9):
    """
    Generate forecasts for all states at a specific hour.
    
    Args:
        data: DataFrame containing load data
        fuzzy_system: Fuzzy logic system
        hour_to_forecast: Hour to forecast for (default: 9 AM)
        
    Returns:
        DataFrame with forecast results
    """
    try:
        states = data['State'].unique()
        forecasts = {}
        
        for state in states:
            print(f"\nProcessing forecasts for {state}")
            
            # Get previous load value (from the previous hour)
            state_data = data[data['State'] == state]
            prev_hour = hour_to_forecast - 1 if hour_to_forecast > 0 else 23
            prev_load_data = state_data[state_data['Hour'] == prev_hour]['Value'].values
            
            if len(prev_load_data) == 0:
                print(f"No data found for {state} at hour {prev_hour}. Skipping.")
                continue
                
            prev_load = prev_load_data[-1]
            
            # Generate forecasts
            hybrid_pred, nn_pred, fuzzy_pred = hybrid_load_forecast(
                data, state, hour_to_forecast, prev_load, fuzzy_system
            )
            
            forecasts[state] = {
                'hybrid': hybrid_pred,
                'neural_network': nn_pred,
                'fuzzy_logic': fuzzy_pred,
                'previous_load': prev_load
            }
        
        # Create a results dataframe
        if not forecasts:
            print("No forecasts generated.")
            return None
            
        results = pd.DataFrame.from_dict(forecasts, orient='index')
        results['percent_change'] = ((results['hybrid'] - results['previous_load']) / results['previous_load'] * 100).round(2)
        
        print("\nLoad Forecast Results:")
        print(results)
        
        # Plot the forecasted values for all states
        plt.figure(figsize=(14, 8))
        plt.bar(results.index, results['hybrid'])
        plt.title(f'Forecasted Load at {hour_to_forecast}:00 for All States')
        plt.ylabel('Load (MW)')
        plt.xticks(rotation=45)
        plt.grid(axis='y')
        plt.tight_layout()
        
        # Save the plot
        output_dir = 'results'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        plt.savefig(f"{output_dir}/all_states_forecast_hour_{hour_to_forecast}.png")
        plt.close()
        
        # Save results to CSV
        results.to_csv(f"{output_dir}/forecast_results_hour_{hour_to_forecast}.csv")
        
        return results
    
    except Exception as e:
        print(f"Error in forecasting all states: {e}")
        return None


In [8]:
def load_data(file_path):
    """
    Load and preprocess the electrical load data.
    
    Args:
        file_path: Path to the data file
        
    Returns:
        Preprocessed DataFrame
    """
    try:
        # Try to load the data
        data = pd.read_csv(file_path)
        
        print(f"Data loaded successfully with {data.shape[0]} rows and {data.shape[1]} columns.")
        
        # Extract hour from datetime
        if 'Datetime' in data.columns:
            data['Datetime'] = pd.to_datetime(data['Datetime'])
            data['Hour'] = data['Datetime'].dt.hour
            data['Date'] = data['Datetime'].dt.date
        
        # Ensure we have the required columns
        if 'State' not in data.columns:
            print("Error: 'State' column not found in the data.")
            return None
            
        if 'Value' not in data.columns:
            print("Error: 'Value' column not found in the data.")
            return None
        
        # Check data types and convert if necessary
        data['Value'] = data['Value'].astype(float)
        
        # Print some basic statistics
        print("\nData summary:")
        print(f"Time period: {data['Datetime'].min()} to {data['Datetime'].max()}")
        print(f"States in the dataset: {data['State'].unique()}")
        print(f"Load value range: {data['Value'].min()} to {data['Value'].max()}")
        
        return data
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def main():
    """
    Main function to run the load forecasting system.
    """
    try:
        print("Starting Load Forecasting System...")
        print(f"Current time: {datetime.datetime.now()}")
        
        # Load the data
        file_path = 'export_data.csv.txt'  # Correct file path with .txt extension
        if not os.path.exists(file_path):
            file_path = input("Enter the path to your data file: ")
        
        data = load_data(file_path)
        if data is None:
            print("Failed to load data. Exiting.")
            return
        
        # Create fuzzy system
        fuzzy_system = create_fuzzy_system()
        if fuzzy_system is None:
            print("Failed to create fuzzy system. Exiting.")
            return
        
        # Display available states
        states = data['State'].unique()
        print(f"\nAvailable states in the dataset: {states}")
        
        # Select a state for detailed evaluation
        state_to_evaluate = 'PUNJAB'  # The state we want to evaluate
        if state_to_evaluate not in states:
            state_to_evaluate = states[0]
            print(f"Selected state not found. Using {state_to_evaluate} instead.")
        
        # Evaluate the model for the selected state
        print(f"\nEvaluating model for {state_to_evaluate}...")
        hybrid_predictions, nn_predictions, fuzzy_predictions = evaluate_model(
            data, state_to_evaluate, fuzzy_system
        )
        
        # Print forecasted values
        if hybrid_predictions:
            print(f"\nForecasted Load Values for {state_to_evaluate}:")
            for i, (hybrid, nn, fuzzy) in enumerate(zip(hybrid_predictions, nn_predictions, fuzzy_predictions)):
                print(f"Hour {i}: Hybrid = {hybrid:.2f}, NN = {nn:.2f}, Fuzzy = {fuzzy:.2f}")
        
        # Generate forecasts for all states at a specific hour
        hour_to_forecast = 9  # 9 AM
        print(f"\nGenerating forecasts for all states at hour {hour_to_forecast}...")
        forecast_results = forecast_all_states(data, fuzzy_system, hour_to_forecast)
        
        print("\nLoad Forecasting System completed successfully!")
        
    except Exception as e:
        print(f"Error in main function: {e}")


In [17]:
def load_data(file_path):
    """
    Load and preprocess the electrical load data.
    
    Args:
        file_path: Path to the data file
        
    Returns:
        Preprocessed DataFrame
    """
    try:
        # Try to load the data
        data = pd.read_csv(file_path)
        
        print(f"Data loaded successfully with {data.shape[0]} rows and {data.shape[1]} columns.")
        
        # Extract hour from datetime
        if 'Datetime' in data.columns:
            data['Datetime'] = pd.to_datetime(data['Datetime'])
            data['Hour'] = data['Datetime'].dt.hour
            data['Date'] = data['Datetime'].dt.date
        
        # Ensure we have the required columns
        if 'State' not in data.columns:
            print("Error: 'State' column not found in the data.")
            return None
            
        if 'Value' not in data.columns:
            print("Error: 'Value' column not found in the data.")
            return None
        
        # Check data types and convert if necessary
        data['Value'] = data['Value'].astype(float)
        
        # Print some basic statistics
        print("\nData summary:")
        print(f"Time period: {data['Datetime'].min()} to {data['Datetime'].max()}")
        print(f"States in the dataset: {data['State'].unique()}")
        print(f"Load value range: {data['Value'].min()} to {data['Value'].max()}")
        
        return data
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None



In [18]:
def main():
    print("Starting Load Forecasting System...")
    print(f"Current time: {datetime.datetime.now()}")
    
    # Load the data
    file_path = 'export_data.csv.txt'  # Update to match your file name
    if not os.path.exists(file_path):
        file_path = input("Enter the path to your data file: ")
    
    data = load_data(file_path)
    if data is None:
        print("Failed to load data. Exiting.")
        return
    
    # Create fuzzy system
    fuzzy_system = create_fuzzy_system()
    if fuzzy_system is None:
        print("Failed to create fuzzy system. Exiting.")
        return
    
    # Display available states
    states = data['State'].unique()
    print(f"\nAvailable states in the dataset: {states}")
    
    # Select a state for detailed evaluation
    state_to_evaluate = 'UTTAR-PRADESH'  # Change this to your desired state
    if state_to_evaluate not in states:
        state_to_evaluate = states[0]
        print(f"Selected state not found. Using {state_to_evaluate} instead.")
    
    # Evaluate the model for the selected state
    print(f"\nEvaluating model for {state_to_evaluate}...")
    hybrid_predictions, nn_predictions, fuzzy_predictions = evaluate_model(
        data, state_to_evaluate, fuzzy_system
    )
    
    # Print forecasted values
    if hybrid_predictions:
        print(f"\nForecasted Load Values for {state_to_evaluate}:")
        for i, (hybrid, nn, fuzzy) in enumerate(zip(hybrid_predictions, nn_predictions, fuzzy_predictions)):
            print(f"Hour {i}: Hybrid = {hybrid:.2f}, NN = {nn:.2f}, Fuzzy = {fuzzy:.2f}")
    
    # Generate forecasts for all states at a specific hour
    hour_to_forecast = 9  # 9 AM
    print(f"\nGenerating forecasts for all states at hour {hour_to_forecast}...")
    forecast_results = forecast_all_states(data, fuzzy_system, hour_to_forecast)
    
    print("\nLoad Forecasting System completed successfully!")

if __name__ == "__main__":
    main()

        

Starting Load Forecasting System...
Current time: 2025-04-07 21:30:44.578263
Data loaded successfully with 29752 rows and 5 columns.

Data summary:
Time period: 2025-02-11 18:37:29.166139 to 2025-03-26 16:21:12.683044
States in the dataset: ['CHANDIGARH' 'HARYANA' 'HIMACHAL' 'J & K' 'PUNJAB' 'RAJASTHAN'
 'UTTARAKHAND' 'UTTAR-PRADESH']
Load value range: 83.0 to 21262.0
Fuzzy logic system created successfully!

Available states in the dataset: ['CHANDIGARH' 'HARYANA' 'HIMACHAL' 'J & K' 'PUNJAB' 'RAJASTHAN'
 'UTTARAKHAND' 'UTTAR-PRADESH']

Evaluating model for UTTAR-PRADESH...
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: UTTAR-PRADESH
Train RMSE: 403.35, Test RMSE: 410.83
Train R²: 0.9700, Test R²: 0.9627
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: UTTAR-PRADESH
Train RMSE: 408.72, Test RMSE: 413.83
Train R²: 0.9692, Test R²: 0.9622
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
9

In [21]:
def visualize_fuzzy_sets():
    """
    Visualize the fuzzy sets for better understanding.
    """
    try:
        # Create a new figure
        fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(10, 8))
        
        # Define universes
        time_universe = np.arange(0, 24, 0.1)
        load_universe = np.arange(0, 20000, 100)
        
        # Define membership functions for time of day
        midnight = fuzz.trimf(time_universe, [0, 3, 6])
        morning = fuzz.trimf(time_universe, [5, 8, 11])
        noon = fuzz.trimf(time_universe, [10, 12, 14])
        afternoon = fuzz.trimf(time_universe, [13, 15, 17])
        evening = fuzz.trimf(time_universe, [16, 19, 22])
        night = fuzz.trimf(time_universe, [21, 23, 24])
        
        # Define membership functions for load
        very_low = fuzz.trimf(load_universe, [0, 2000, 4000])
        low = fuzz.trimf(load_universe, [3000, 5000, 7000])
        average = fuzz.trimf(load_universe, [6000, 8000, 10000])
        high = fuzz.trimf(load_universe, [9000, 12000, 15000])
        very_high = fuzz.trimf(load_universe, [14000, 17000, 20000])
        
        # Plot time of day membership functions
        ax0.plot(time_universe, midnight, 'b', linewidth=1.5, label='Midnight')
        ax0.plot(time_universe, morning, 'g', linewidth=1.5, label='Morning')
        ax0.plot(time_universe, noon, 'r', linewidth=1.5, label='Noon')
        ax0.plot(time_universe, afternoon, 'c', linewidth=1.5, label='Afternoon')
        ax0.plot(time_universe, evening, 'm', linewidth=1.5, label='Evening')
        ax0.plot(time_universe, night, 'y', linewidth=1.5, label='Night')
        ax0.set_title('Time of Day Membership Functions')
        ax0.legend()
        
        # Plot previous load membership functions
        ax1.plot(load_universe, very_low, 'b', linewidth=1.5, label='Very Low')
        ax1.plot(load_universe, low, 'g', linewidth=1.5, label='Low')
        ax1.plot(load_universe, average, 'r', linewidth=1.5, label='Average')
        ax1.plot(load_universe, high, 'c', linewidth=1.5, label='High')
        ax1.plot(load_universe, very_high, 'm', linewidth=1.5, label='Very High')
        ax1.set_title('Previous Load Membership Functions')
        ax1.legend()
        
        # Plot forecasted load membership functions
        ax2.plot(load_universe, very_low, 'b', linewidth=1.5, label='Very Low')
        ax2.plot(load_universe, low, 'g', linewidth=1.5, label='Low')
        ax2.plot(load_universe, average, 'r', linewidth=1.5, label='Average')
        ax2.plot(load_universe, high, 'c', linewidth=1.5, label='High')
        ax2.plot(load_universe, very_high, 'm', linewidth=1.5, label='Very High')
        ax2.set_title('Forecasted Load Membership Functions')
        ax2.legend()
        
        # Adjust layout and save
        plt.tight_layout()
        output_dir = 'results'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        plt.savefig(f"{output_dir}/fuzzy_membership_functions.png")
        plt.close()
        
        print("Fuzzy membership functions visualized and saved.")
    
    except Exception as e:
        print(f"Error visualizing fuzzy sets: {e}")

def analyze_data(data):
    """
    Perform exploratory data analysis on the dataset.
    
    Args:
        data: DataFrame containing load data
    """
    try:
        # Create output directory
        output_dir = 'results'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Basic statistics
        print("\nData Summary:")
        print(data.describe())
        
        # Check for missing values
        missing_values = data.isnull().sum()
        print("\nMissing Values:")
        print(missing_values)
        
        # Plot load by state
        plt.figure(figsize=(12, 6))
        data.boxplot(column='Value', by='State')
        plt.title('Load Distribution by State')
        plt.ylabel('Load (MW)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/load_by_state.png")
        plt.close()
        
        # Plot load by hour
        if 'Hour' in data.columns:
            hourly_avg = data.groupby(['Hour', 'State'])['Value'].mean().unstack()
            plt.figure(figsize=(14, 8))
            hourly_avg.plot(marker='o')
            plt.title('Average Load by Hour for Each State')
            plt.xlabel('Hour of Day')
            plt.ylabel('Average Load (MW)')
            plt.grid(True)
            plt.legend(title='State')
            plt.tight_layout()
            plt.savefig(f"{output_dir}/load_by_hour.png")
            plt.close()
        
        # Time series plot for each state
        states = data['State'].unique()
        for state in states:
            state_data = data[data['State'] == state]
            if 'Datetime' in state_data.columns:
                plt.figure(figsize=(14, 6))
                plt.plot(state_data['Datetime'], state_data['Value'])
                plt.title(f'Load Time Series for {state}')
                plt.xlabel('Date/Time')
                plt.ylabel('Load (MW)')
                plt.grid(True)
                plt.tight_layout()
                plt.savefig(f"{output_dir}/{state}_time_series.png")
                plt.close()
        
        print("Data analysis completed and visualizations saved.")
    
    except Exception as e:
        print(f"Error in data analysis: {e}")


In [23]:
def extended_main():
    """
    Extended main function with additional features.
    """
    try:
        print("Starting Advanced Load Forecasting System...")
        print(f"Current time: {datetime.datetime.now()}")
        
        # Load the data
        file_path = 'export_data.csv.txt'  # Change this to your actual file path
        if not os.path.exists(file_path):
            file_path = input("Enter the path to your data file: ")
        
        data = load_data(file_path)
        if data is None:
            print("Failed to load data. Exiting.")
            return
        
        # Analyze the data
        print("\nPerforming data analysis...")
        analyze_data(data)
        
        # Create fuzzy system
        fuzzy_system = create_fuzzy_system()
        if fuzzy_system is None:
            print("Failed to create fuzzy system. Exiting.")
            return
        
        # Visualize fuzzy sets
        print("\nVisualizing fuzzy membership functions...")
        visualize_fuzzy_sets()
        
        # Display available states
        states = data['State'].unique()
        print(f"\nAvailable states in the dataset: {states}")
        
        # Menu for user interaction
        while True:
            print("\n===== Load Forecasting System Menu =====")
            print("1. Evaluate model for a specific state")
            print("2. Generate forecasts for all states")
            print("3. Visualize fuzzy membership functions")
            print("4. Analyze data")
            print("5. Exit")
            
            choice = input("\nEnter your choice (1-5): ")
            
            if choice == '1':
                # Select a state for detailed evaluation
                print(f"\nAvailable states: {states}")
                state_to_evaluate = input("Enter state name (or press Enter for default): ")
                
                if not state_to_evaluate:
                    state_to_evaluate = 'PUNJAB'  # Default state
                
                if state_to_evaluate not in states:
                    print(f"State '{state_to_evaluate}' not found. Available states: {states}")
                    continue
                
                # Evaluate the model for the selected state
                print(f"\nEvaluating model for {state_to_evaluate}...")
                hybrid_predictions, nn_predictions, fuzzy_predictions = evaluate_model(
                    data, state_to_evaluate, fuzzy_system
                )
                
                # Print forecasted values
                if hybrid_predictions:
                    print(f"\nForecasted Load Values for {state_to_evaluate}:")
                    for i, (hybrid, nn, fuzzy) in enumerate(zip(hybrid_predictions, nn_predictions, fuzzy_predictions)):
                        print(f"Hour {i}: Hybrid = {hybrid:.2f}, NN = {nn:.2f}, Fuzzy = {fuzzy:.2f}")
            
            elif choice == '2':
                # Generate forecasts for all states at a specific hour
                hour_input = input("Enter hour to forecast (0-23, default is 9): ")
                try:
                    hour_to_forecast = int(hour_input) if hour_input else 9
                    if hour_to_forecast < 0 or hour_to_forecast > 23:
                        print("Hour must be between 0 and 23. Using default (9).")
                        hour_to_forecast = 9
                except ValueError:
                    print("Invalid input. Using default hour (9).")
                    hour_to_forecast = 9
                
                print(f"\nGenerating forecasts for all states at hour {hour_to_forecast}...")
                forecast_results = forecast_all_states(data, fuzzy_system, hour_to_forecast)
            
            elif choice == '3':
                # Visualize fuzzy sets
                print("\nVisualizing fuzzy membership functions...")
                visualize_fuzzy_sets()
            
            elif choice == '4':
                # Analyze data
                print("\nPerforming data analysis...")
                analyze_data(data)
            
            elif choice == '5':
                print("\nExiting Load Forecasting System. Goodbye!")
                break
            
            else:
                print("Invalid choice. Please enter a number between 1 and 5.")
    
    except Exception as e:
        print(f"Error in extended main function: {e}")

# Run the extended main function if this script is executed directly
if __name__ == "__main__":
    # Uncomment the line below to use the extended interactive version
    extended_main()
    
    # Use the basic version by default
    #main()


Starting Advanced Load Forecasting System...
Current time: 2025-04-07 22:28:30.552663
Data loaded successfully with 29752 rows and 5 columns.

Data summary:
Time period: 2025-02-11 18:37:29.166139 to 2025-03-26 16:21:12.683044
States in the dataset: ['CHANDIGARH' 'HARYANA' 'HIMACHAL' 'J & K' 'PUNJAB' 'RAJASTHAN'
 'UTTARAKHAND' 'UTTAR-PRADESH']
Load value range: 83.0 to 21262.0

Performing data analysis...

Data Summary:
              Value     Timestamp                       Datetime          Hour
count  29752.000000  2.975200e+04                          29752  29752.000000
mean    5895.642444  1.741091e+09  2025-03-04 18:00:59.813930496     11.383436
min       83.000000  1.739279e+09     2025-02-11 18:37:29.166139      0.000000
25%     1583.000000  1.740147e+09  2025-02-21 19:39:02.859421952      5.000000
50%     3021.000000  1.741024e+09  2025-03-03 23:23:28.841452032     11.000000
75%     9467.000000  1.742058e+09  2025-03-15 22:33:06.584214016     17.000000
max    21262.000000  1.


Enter your choice (1-5):  1



Available states: ['CHANDIGARH' 'HARYANA' 'HIMACHAL' 'J & K' 'PUNJAB' 'RAJASTHAN'
 'UTTARAKHAND' 'UTTAR-PRADESH']


Enter state name (or press Enter for default):  



Evaluating model for PUNJAB...
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: PUNJAB
Train RMSE: 184.77, Test RMSE: 171.34
Train R²: 0.9844, Test R²: 0.9799
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: PUNJAB
Train RMSE: 212.13, Test RMSE: 216.85
Train R²: 0.9795, Test R²: 0.9678
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: PUNJAB
Train RMSE: 184.82, Test RMSE: 171.29
Train R²: 0.9844, Test R²: 0.9799
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: PUNJAB
Train RMSE: 179.04, Test RMSE: 166.04
Train R²: 0.9854, Test R²: 0.9811
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: PUNJAB
Train RMSE: 188.30, Test RMSE: 176.32
Train R²: 0.9838, Test R²: 0.9


Enter your choice (1-5):  2
Enter hour to forecast (0-23, default is 9):  11



Generating forecasts for all states at hour 11...

Processing forecasts for CHANDIGARH
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: CHANDIGARH
Train RMSE: 5.00, Test RMSE: 3.75
Train R²: 0.9843, Test R²: 0.9836
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Processing forecasts for HARYANA
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: HARYANA
Train RMSE: 206.92, Test RMSE: 177.77
Train R²: 0.9624, Test R²: 0.9061
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Processing forecasts for HIMACHAL
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: HIMACHAL
Train RMSE: 36.72, Test RMSE: 33.71
Train R²: 0.9855, Test R²: 0.9765
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Processing forecasts for J & K
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
State: J & K
Train RMSE: 64.33, Test RMSE: 66.12
Train R²: 0.9588, Test R²: 0.9164
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

Proc

KeyboardInterrupt: Interrupted by user

<Figure size 1200x600 with 0 Axes>

<Figure size 1400x800 with 0 Axes>